# Homework (deadline 16.12.2022 13:44:59)

Write solutions for the homework exercises in this notebook. Once the work is done download the notebook file (`File > Download .ipynb`) rename it properly so it follows a template `HW2_<SURNAME>_<NAME>.ipynb` and send the file to me. **Please also attach all files you will be asked to produce in this homework**. My email address is as follows: 

* <m.biesaga@uw.edu.pl>

Remember that you can contact me via email if you have any problems. Moreover, you can also visit me in the ISS on the fourth floor (room 415). Usually, I am there from 11ish but please let me know in advance if you are coming because I might be busy. 


## Task 1 (15 points)
You all should be able to get the file `links.jl` on our [Google Drive](https://drive.google.com/file/d/1qkIUwxMZmI-CMXUaBfLMTj7OChmP9cOW/view?usp=share_link). It contains 30 links to EURACTIV, The Guardian, and The Parliament Magazine articles. Your task is to:

* Read `links.jl` to Python (Google Colab)
* Scrap the articles from the given links
* Write out the results into `HW2_<SURNAME>_<NAME>.jl`

From every website you should get the following information:

* Title of the article
* Author of the text (if present, if not the value should be None)
* Lead of the article (summary)
* Date of publication
* Text of the article

Additionally, you should count the number of words in the given article and extract only relevant information from the `links.jl`.

*Reminder:* A JSON line file contains multiple JSONs. As the name suggests each line contains one. Therefore, each line should look more or less like that:

```python
{ 
	"title" : "Why are Namedays better than Birthdays?",
    "author" : "M. Biesaga",
    "date" : "06.12.2019",
    "lead" : "Scientists from one of the best Universities in the U.S. proved that the discussion about birthdays and namedays is finally over.",
    "content" : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer eget sapien nisi. In placerat nisl felis, vel porttitor odio aliquam quis. Nulla a facilisis arcu. Suspendisse potenti. Interdum et malesuada fames ac ante ipsum primis in faucibus. Duis et semper urna. Mauris sit amet enim ex. Integer eget ultricies enim, at tristique sem. In eu eros nisl. Nulla vitae pretium risus, quis vestibulum dui. Nullam vitae dapibus quam. Maecenas commodo dictum ex, id vestibulum ex volutpat interdum. Cras tempor diam non urna auctor, vitae dignissim tortor tristique. Nunc consectetur mauris non lorem luctus aliquam. Mauris vitae ligula orci.",
	"source" : "Journal of Scientific Science",
	"fb" : { 
		        "likes" : 112,
	            "shares" : 2,
			    "comments" : 43
	},
	"length" : 98 
}
```
*IMPORTANT*: If you are going to send requests in a loop please don't send more than 1 request per second. You can use `time.sleep(1)` from the `time` module to halt your code for 1 second. For example, in a loop it would look something like that:
```python
for line in links:
	## Send request
	time.sleep(1)
```
It will perform the code in a loop and at the end of each iteration, it will wait for a second.

In [ ]:
import json
import requests as rq
from bs4 import BeautifulSoup
import time

## YOUR CODE